In [33]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Data Load and other util function

In [124]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()

#     understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    for item in ['Kind','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    return df

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

def get_spearmanr(df):
    df = df.drop('Bugs',axis = 1)
    file_corr = []
    for name in df.Name.unique():
        sub_df = df[df['Name'] == name]
        if sub_df.shape[0] < 2:
            continue
        sub_df = sub_df.drop('Name',axis = 1)
        rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)
        file_corr.append(np.median(rho[:,:int(len(rho)/2)]))
    return file_corr

# Getting List of projects

In [121]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

# Run the experiment

In [ ]:
project_corr = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'Pokemap':
#             continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        df = load_data(project)
        corr = get_spearmanr(df)
        project_corr[project] = corr
        print(np.median(corr))
    except Exception as e:
        print(e)
        continue

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
0.9952894018179659
+++++++++++++++++   qpython  +++++++++++++++++
1.0
+++++++++++++++++   friendlychat-android  +++++++++++++++++
0.9722682462009827
+++++++++++++++++   paho.mqtt.android  +++++++++++++++++
1.0
+++++++++++++++++   paho.mqtt.java  +++++++++++++++++
1.0
+++++++++++++++++   android-docs-samples  +++++++++++++++++
1.0
+++++++++++++++++   Toasty  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   android-mvvm-architecture  +++++++++++++++++
1.0
+++++++++++++++++   recipes-rss  +++++++++++++++++
1.0
+++++++++++++++++   HoloGraphLibrary  +++++++++++++++++
1.0
+++++++++++++++++   HorizontalWheelView  +++++++++++++++++
1.0
+++++++++++++++++   OpenMemories-Tweak  +++++++++++++++++
1.0
+++++++++++++++++   HoldingButton  +++++++++++++++++
0.9997536005684914
+++++++++++++++++   ExpandableLayout  +++++++++++++++++
1.0
+++++++++++++++++   RecyclerView-MultipleViewTypesAdapter  +++++++++++++++++
0.9999999999999999
++++++

0.9999999999999999
+++++++++++++++++   PowerMenu  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   TagsEditText  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   AutoUpdateProject  +++++++++++++++++
1.0
+++++++++++++++++   maven-plugins  +++++++++++++++++
1.0
+++++++++++++++++   CameraFragment  +++++++++++++++++
1.0
+++++++++++++++++   carbon-apimgt  +++++++++++++++++
1.0
+++++++++++++++++   android-signaturepad  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   junit5-samples  +++++++++++++++++
1.0
+++++++++++++++++   SoundRecorder  +++++++++++++++++
1.0
+++++++++++++++++   jd-gui  +++++++++++++++++
1.0
+++++++++++++++++   aws-apigateway-importer  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   Long-Shadows  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   SmartIM4IntelliJ  +++++++++++++++++
1.0
+++++++++++++++++   SelectorInjection  +++++++++++++++++
0.9944422147310641
+++++++++++++++++   Biscuit  +++++++++++++++++
1.0
+++++++++++++++++   Badge

1.0
+++++++++++++++++   DataflowTemplates  +++++++++++++++++
1.0
+++++++++++++++++   the-app  +++++++++++++++++
1.0
+++++++++++++++++   glide-transformations  +++++++++++++++++
0.9829592943035645
+++++++++++++++++   X2C  +++++++++++++++++
1.0
+++++++++++++++++   Android-DirectoryChooser  +++++++++++++++++
1.0
+++++++++++++++++   jdeferred  +++++++++++++++++
1.0
+++++++++++++++++   RxBluetooth  +++++++++++++++++
1.0
+++++++++++++++++   react-native-calendar-events  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   cordova-plugin-admob-free  +++++++++++++++++
0.9999999999999999
+++++++++++++++++   sofa-tracer  +++++++++++++++++
1.0
+++++++++++++++++   react-native-touch-id  +++++++++++++++++
1.0
+++++++++++++++++   linuxdeploy  +++++++++++++++++
1.0
+++++++++++++++++   brickhouse  +++++++++++++++++
1.0
+++++++++++++++++   FinestWebView-Android  +++++++++++++++++
1.0
+++++++++++++++++   tut-spring-boot-oauth2  +++++++++++++++++
1.0
+++++++++++++++++   android-viewflow  ++++++++++++

In [92]:
project_corr

{'Pokemap': 1.0}

In [87]:
df = df.drop('Bugs',axis = 1)
for name in df.Name.unique():
    sub_df = df[df['Name'] == name]
    sub_df = sub_df.drop('Name',axis = 1)
    rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)
    print(np.median(rho))

1.0
1.0
0.982977093922288
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999

In [51]:
sub_df.values.shape

(7, 45)

In [43]:
x2n=np.random.randn(100,60)
y2n=np.random.randn(100,60)

In [82]:
rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)

In [109]:
rho[:,:int(len(rho)/2)]

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])